# Differentiable Programming


In [0]:
%disableCompletions


In [0]:

func myFunction(_ a: Double, _ b: Double) -> Double {
    return (a * b) + (a / b) * a
}

In [0]:
let b = 1.3
gradient(at: 0.5) { a in myFunction(a, b) }

In [0]:
gradient(at: 0.5, -0.5) { a, b in myFunction(a, b) }

In [0]:
let result = valueWithGradient(at: 0.5, -0.5) { a, b in myFunction(a, b) }

print(result)

print("The value is: \(result.value)")

print("The gradients are: \(result.gradient)")

You can just use normal types like `Double` and `Float`.

# Custom differential data types


In [0]:
struct Point: Differentiable {
    var x, y: Double

    func dot(_ other: Point) -> Double {
        return x * other.x + y * other.y
    }

    func myHelperFunction() -> Int {
        return Int(x) + Int(y)
    }
}

// Compute gradients
print(gradient(at: Point(x: 1, y: 2)) { p in p.dot(p) })

Recall, magnitude of point $P$ is defined as $\left|P\right| = \sqrt{x^2 + y^2}$



In [0]:
import Glibc

In [0]:
sqrt(4)

In [0]:
extension Point {
    @differentiable
    func magnitude() -> Double {
        return sqrt(x * x + y * y)
    }
}

Custom derivative for `sqrt`:

Recall, $\frac{d}{dx}\sqrt{x} = \frac{d}{dx}x^{\frac{1}{2}} = \frac{1}{2}x^{-\frac{1}{2}} = \frac{1}{2 \sqrt{x}}$

In [0]:
func mySqrt(_ x: Double) -> Double {
    return sqrt(x)
}

@differentiating(mySqrt)
func mySqrtDerivative(_ x: Double) -> (value: Double,
                                       pullback: (Double) -> Double) {
    return (value: mySqrt(x),
            pullback: { chain in (chain / (2 * mySqrt(x))) })
}

In [0]:
// A silly function taking two different differentiable types.
@differentiable
func silly(_ a: Point, _ b: Point, by scale: Double) -> Double {
    return a.dot(b) * b.magnitude() + a.magnitude() * scale
}

In [0]:
let x = Point(x: 3.4, y: -1.7)

In [0]:
let grad = gradient(at: Point(x: 1.0, y: 0)) { p in
    silly(p, x, by: 4.2)
}
print(grad)

In [0]:
let y = Point(x: 1.0, y: -1.0)
let (dx, dy) = gradient(at: x, y) { x, y in silly(x, y, by: 1.3) }
print(dx, dy, separator: "\n")

In [0]:
let (val, grad) = valueWithGradient(at: x, y) { x, y in
    silly(x, y, by: 3.14159)
}
print("The value of silly is: \(val)")
print("The gradients with respect to x is: \(grad.0).")
print("The gradients with respect to y is: \(grad.1).")

# Printing out values in the backward pass


In [0]:
typealias Identity = @differentiable (Double) -> Double
let gradientDebugger: Identity = differentiableFunction { input in
    return (input,
            { gradient in print("Gradient: \(gradient)"); return gradient })
}